---
title: "Presidential Approval Ratings Over Time"
format: 
  html:
    embed-resources: true
    include-in-header:
      - text: |
          <script src="https://cdnjs.cloudflare.com/ajax/libs/highcharts/11.2.0/highcharts.js"></script>
          <script src="https://cdnjs.cloudflare.com/ajax/libs/highcharts/11.2.0/highcharts-more.js"></script>
          <script src="https://code.highcharts.com/modules/grid-axis.js"></script>
          <script src="https://cdnjs.cloudflare.com/ajax/libs/highcharts/11.2.0/modules/exporting.js"></script>
          <script src="https://cdnjs.cloudflare.com/ajax/libs/highcharts/11.2.0/modules/offline-exporting.js"></script>
          <script src="https://cdnjs.cloudflare.com/ajax/libs/highcharts/11.2.0/modules/export-data.js"></script>
          <script src="https://cdnjs.cloudflare.com/ajax/libs/highcharts/11.2.0/modules/accessibility.js"></script>
          <script src="https://code.highcharts.com/modules/pattern-fill.js"></script>
          <script src="https://cdnjs.cloudflare.com/ajax/libs/html2canvas/1.4.1/html2canvas.min.js"></script>
---

In [ ]:
#| echo: false
#| output: asis

import polars as pl
import json

def process_presidential_data(df):
    # Get unique presidents
    unique_presidents = df.select('president').unique().to_series().to_list()
    presidents_data = []
    
    for president in unique_presidents:
        # Filter data for current president
        president_df = df.filter(pl.col('president') == president)
        
        # Get the first row's years value for this president
        years = president_df.select('years').row(0)[0]
        
        # Get values in order of year_in_term
        sorted_df = president_df.sort('year_in_term')
        approval_values = sorted_df.select('approval').to_series().to_list()
        disapproval_values = sorted_df.select('disapproval').to_series().to_list()
        change_status_values = sorted_df.select('change_status').to_series().to_list()
        
        # Create range data for the area between lines
        range_data = [[apr, dis] for apr, dis in zip(approval_values, disapproval_values)]
        
        # Create clean ID from president name
        clean_id = president.lower().replace(' ', '').replace('.', '')
        
        president_data = {
            'id': clean_id,
            'name': president,
            'years': years,
            'approval': approval_values,
            'disapproval': disapproval_values,
            'range': range_data,
            'change_status': change_status_values
        }
        presidents_data.append(president_data)
    
    return presidents_data

# Read and process the data
df = pl.read_csv('presidential_approval.csv')

df_with_diff = df.with_columns([
    pl.struct(['approval', 'disapproval'])
    .map_elements(
        lambda x: 
            f"Approval +{x['approval'] - x['disapproval']}%" 
            if x['approval'] > x['disapproval']
            else f"Disapproval +{x['disapproval'] - x['approval']}%",
        return_dtype=pl.Utf8  # Specify return type as string
    ).alias('change_status')
])

df_with_diff = df_with_diff.sort(['president', 'year_in_term'])

chart_data = process_presidential_data(df_with_diff)

# Convert to JSON and output as JavaScript
print("<script>")
print(f"const multi_chartData = {json.dumps(chart_data)};")
print("</script>")

```{=html}
<div style="padding: 20px;">
    <div id="export-container" style="background-color: white;">
        <!-- Title section -->
        <div style="text-align: center; padding: 20px 0 30px 0;">
            <h1 style="margin: 0; font-size: 2em; color: #333;">Presidential Approval Ratings</h1>
            <p style="margin: 10px 0 0 0; font-size: 1.1em; color: #666;">Tracking Presidential Performance Through the Years</p>
        </div>
        
        <!-- Charts grid -->
        <div id="chartsContainer" style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 20px;">
            <div id="donaldtrump" style="min-width: 300px;"></div>
            <div id="barackobama" style="min-width: 300px;"></div>
            <div id="georgewbush" style="min-width: 300px;"></div>
            <div id="billclinton" style="min-width: 300px;"></div>
            <div id="georgehwbush" style="min-width: 300px;"></div>
            <div id="ronaldreagan" style="min-width: 300px;"></div>
            <div id="jimmycarter" style="min-width: 300px;"></div>
            <div id="geraldford" style="min-width: 300px;"></div>
            <div id="richardnixon" style="min-width: 300px;"></div>
        </div>
    </div>
    
    <!-- Button container -->
    <div style="text-align: center; padding: 20px;">
        <button onclick="exportCharts()" id="export-button" style="padding: 10px 20px; background-color: #4CAF50; color: white; border: none; border-radius: 4px; cursor: pointer;">
            Export as PNG
        </button>
    </div>
</div>

<script>
// Add event listener to the export button
document.getElementById('export-button').addEventListener('click', exportCharts);

function exportCharts() {
    const container = document.getElementById('export-container');
    
    html2canvas(container, {
        backgroundColor: 'white',
        scale: 2, // Increase quality
        logging: true,
        onclone: function(clonedDoc) {
            // Any modifications to the cloned document before capture
        }
    }).then(canvas => {
        // Convert canvas to PNG and trigger download
        const link = document.createElement('a');
        link.download = 'presidential_approval_charts.png';
        link.href = canvas.toDataURL('image/png');
        link.click();
    });
}

// Create charts
document.addEventListener('DOMContentLoaded', function() {
    multi_chartData.forEach(president => {        
        Highcharts.chart(president.id, {
            exporting: { enabled: false },
            title: {
                text: `${president.name}<br><span style="font-size: 12px; color: #666;">${president.years}</span>`,
                useHTML: true
            },
            yAxis: {
                title: { text: 'Rating (%)' },
                min: 20,
                max: 80,
                gridLineWidth: 1
            },
            xAxis: {
                title: { text: 'Years' },
                categories: ['0', '1', '2', '3', '4'],
                gridLineWidth: 1
            },
            legend: {
                align: 'center',
                verticalAlign: 'bottom'
            },
            plotOptions: {
                series: {
                    marker: {
                        enabled: false,
                        radius: 3
                    }
                },
                spline: {
                    tension: 0.4,
                    step: false
                }
            },
            tooltip: {
                shared: true,
                crosshairs: true,
                useHTML: true,
                formatter: function() {
                    let approveValue, disapproveValue;
                    this.points.forEach(point => {
                        if (point.series.name === 'Approval') {
                            approveValue = point.y;
                        } else if (point.series.name === 'Disapproval') {
                            disapproveValue = point.y;
                        }
                    });
                    
                    const difference = approveValue - disapproveValue;
                    const differenceText = difference > 0 ?
                        `Approval +${difference.toFixed(1)}%` :
                        `Disapproval +${(-difference).toFixed(1)}%`;
                    const differenceColor = difference > 0 ? '#059669' : '#EC4899';
                    
                    return `
                        <div style="font-family: sans-serif;">
                            <div style="font-size: 0.9em; color: #666; margin-bottom: 8px;">Year ${this.x}</div>
                            <div style="margin-bottom: 4px;">
                                <span style="color: #059669">●</span> Approval:
                                <b>${approveValue?.toFixed(1)}%</b>
                            </div>
                            <div style="margin-bottom: 8px;">
                                <span style="color: #EC4899">●</span> Disapproval:
                                <b>${disapproveValue?.toFixed(1)}%</b>
                            </div>
                            <div style="padding-top: 8px; border-top: 1px solid #eee;">
                                <span style="color: ${differenceColor}; font-weight: 500">
                                    ${differenceText}
                                </span>
                            </div>
                        </div>
                    `;
                }
            },
            series: [{
                name: 'Approval',
                type: 'spline',
                data: president.approval,
                color: '#059669',
                lineWidth: 2,
                zIndex: 2,
                marker: {
                    enabled: false,
                    radius: 3
                }
            }, {
                name: 'Disapproval',
                type: 'spline',
                data: president.disapproval,
                color: '#EC4899',
                lineWidth: 2,
                zIndex: 2,
                marker: {
                    enabled: false,
                    radius: 3
                }
            }, {
                type: 'areasplinerange',
                name: 'Range',
                data: president.range,
                color: '#6B7280',
                fillOpacity: 0.1,
                lineWidth: 0,
                showInLegend: false,
                enableMouseTracking: false,
                zIndex: 1
            }],
            credits: {
                enabled: false
            }
        });
    });
});
</script>

<style>
#export-button:hover {
    background-color: #45a049;
}
</style>
```